In [1]:
import pandas as pd
import os
import pathlib
#os.chdir('/home/adurden/Ari/AMS526/AMS561FP')

In [2]:
b3lyp = pd.read_csv('b3lyp_eprec6.csv')
hf = pd.read_csv('hf_eprec6.csv')
lda = pd.read_csv('lda_eprec6.csv')
pbe0 = pd.read_csv('pbe0_eprec6.csv')

In [3]:
# column of names is named Unnamed for some reason
print(pbe0['Unnamed: 0'])
print(lda['Unnamed: 0'])

0      bicyclo[3.1.0]hexane
1                 phosphine
2         beryllium_sulfide
3             lithium_oxide
4           carbon_monoxide
               ...         
220            cyclopropene
221             cyclobutane
222              silylene_s
223    aluminum_monohydride
224         beryllium_oxide
Name: Unnamed: 0, Length: 225, dtype: object
0      bicyclo[3.1.0]hexane
1                 phosphine
2         beryllium_sulfide
3             lithium_oxide
4           carbon_monoxide
               ...         
220            cyclopropene
221             cyclobutane
222              silylene_s
223    aluminum_monohydride
224         beryllium_oxide
Name: Unnamed: 0, Length: 225, dtype: object


In [4]:
print(pbe0['Energy'])

0     -234.448220
1     -343.028857
2     -412.752433
3      -82.619718
4     -113.237996
          ...    
220   -116.526064
221   -157.084428
222   -290.496811
223   -242.867793
224    -89.829275
Name: Energy, Length: 225, dtype: float64


In [5]:
# the whole csv file (dipoles, forces, wall time etc)
print(pbe0)

               Unnamed: 0      Energy  Iterations  Dipole Moment  \
0    bicyclo[3.1.0]hexane -234.448220          12   7.575750e-02   
1               phosphine -343.028857          10   2.507087e-01   
2       beryllium_sulfide -412.752433          14   2.164581e+00   
3           lithium_oxide  -82.619718          11   2.568827e+00   
4         carbon_monoxide -113.237996           9   3.864714e-02   
..                    ...         ...         ...            ...   
220          cyclopropene -116.526064          10   1.839676e-01   
221           cyclobutane -157.084428          10   7.186659e-07   
222            silylene_s -290.496811          10   8.775786e-02   
223  aluminum_monohydride -242.867793          10   2.059471e-02   
224       beryllium_oxide  -89.829275          13   2.573871e+00   

                                                Eigens  \
0    [-10.22198465, -10.22168891, -10.21485057, -10...   
1    [-77.20463688, -6.60599131, -4.74144868, -4.74...   
2    [-88

In [6]:
# go through input files in geometry folder and parse geometry block into dictonary mols
#
# get relative path assuming geometries are in separate folder in the same directory
path = os.path.abspath(os.getcwd())
geometries_path = os.path.join(path, 'geometries')
mols = {}
# if relative path doesn't work use absolute path instead
#for filename in os.scandir("/home/hannes/Documents/SBU/AMS561/ML/geometries"):
for filename in os.scandir(geometries_path):
    # parse name of molecule (filename) for key
    # does not distinguish files - make sure ONLY mdns files are in director
    # e.g. no swap files from editors
    name = str(filename).split()[1]
    name = name.replace('.mdns', '')
    name = name.replace("'", "")
    name = name.replace(">", "")
    if filename.is_file():
        f = open(filename, "r")
        # list of lists containing [letter, x, y, z]
        mol = []
        # the format is
        # geometry
        # units angstrom
        #  H 0 0 0
        #  ...
        # end
        while True:
            line = f.readline()
            # skip the angstrom and geometry lines (not very clean but works for now)
            if "geometry" in line:
                line = f.readline()
                #if "units angstrom" in line:
                 #   f.readline()
                while True:
                    line = f.readline()
                    if line == "end":
                        break
                    mol.append(line.split())
                mols[name] = mol 
                #print(len(mol))       
            if not line:
                break

In [7]:
print(mols["water"])

[['O', '0.000000000000', '0.000000000000', '0.117300000000'], ['H', '0.000000000000', '0.757200000000', '-0.469200000000'], ['H', '0.000000000000', '-0.757200000000', '-0.469200000000']]


In [8]:
# print a molecule
mol_name = "phosphine"
for i in mols[mol_name]:
    print(i)

['P', '0.000000000000', '0.000000000000', '0.000000000000']
['H', '0.000000000000', '-1.193200000000', '-0.771700000000']
['H', '1.033300000000', '0.596600000000', '-0.771700000000']
['H', '-1.033300000000', '0.596600000000', '-0.771700000000']


Define a dictionary with symbols and atomic numbers of each element


In [9]:
elements = {
    "H": 1,
    "He": 2,
    "Li": 3,
    "Be": 4,
    "B": 5,
    "C": 6,
    "N": 7,
    "O": 8,
    "F": 9,
    "Ne": 10,
    "Na": 11,
    "Mg": 12,
    "Al": 13,
    "Si": 14,
    "P": 15,
    "S": 16,
    "Cl": 17,
    "Ar": 18,
    "K": 19,
    "Ca": 20,
    "Sc": 21,
    "Ti": 22,
    "V": 23,
    "Cr": 24,
    "Mn": 25,
    "Fe": 26,
    "Co": 27,
    "Ni": 28,
    "Cu": 29,
    "Zn": 30,
    "Ga": 31,
    "Ge": 32,
    "As": 33,
    "Se": 34,
    "Br": 35,
    "Kr": 36,
    "Rb": 37,
    "Sr": 38,
    "Y": 39,
    "Zr": 40,
    "Nb": 41,
    "Mo": 42,
    "Tc": 43,
    "Ru": 44,
    "Rh": 45,
    "Pd": 46,
    "Ag": 47,
    "Cd": 48,
    "In": 49,
    "Sn": 50,
    "Sb": 51,
    "Te": 52,
    "I": 53,
    "Xe": 54,
    "Cs": 55,
    "Ba": 56,
    "La": 57,
    "Ce": 58,
    "Pr": 59,
    "Nd": 60,
    "Pm": 61,
    "Sm": 62,
    "Eu": 63,
    "Gd": 64,
    "Tb": 65,
    "Dy": 66,
    "Ho": 67,
    "Er": 68,
    "Tm": 69,
    "Yb": 70,
    "Lu": 71,
    "Hf": 72,
    "Ta": 73,
    "W": 74,
    "Re": 75,
    "Os": 76,
    "Ir": 77,
    "Pt": 78,
    "Au": 79,
    "Hg": 80
}

print(elements)


{'H': 1, 'He': 2, 'Li': 3, 'Be': 4, 'B': 5, 'C': 6, 'N': 7, 'O': 8, 'F': 9, 'Ne': 10, 'Na': 11, 'Mg': 12, 'Al': 13, 'Si': 14, 'P': 15, 'S': 16, 'Cl': 17, 'Ar': 18, 'K': 19, 'Ca': 20, 'Sc': 21, 'Ti': 22, 'V': 23, 'Cr': 24, 'Mn': 25, 'Fe': 26, 'Co': 27, 'Ni': 28, 'Cu': 29, 'Zn': 30, 'Ga': 31, 'Ge': 32, 'As': 33, 'Se': 34, 'Br': 35, 'Kr': 36, 'Rb': 37, 'Sr': 38, 'Y': 39, 'Zr': 40, 'Nb': 41, 'Mo': 42, 'Tc': 43, 'Ru': 44, 'Rh': 45, 'Pd': 46, 'Ag': 47, 'Cd': 48, 'In': 49, 'Sn': 50, 'Sb': 51, 'Te': 52, 'I': 53, 'Xe': 54, 'Cs': 55, 'Ba': 56, 'La': 57, 'Ce': 58, 'Pr': 59, 'Nd': 60, 'Pm': 61, 'Sm': 62, 'Eu': 63, 'Gd': 64, 'Tb': 65, 'Dy': 66, 'Ho': 67, 'Er': 68, 'Tm': 69, 'Yb': 70, 'Lu': 71, 'Hf': 72, 'Ta': 73, 'W': 74, 'Re': 75, 'Os': 76, 'Ir': 77, 'Pt': 78, 'Au': 79, 'Hg': 80}


In [10]:
import numpy as np
def distance(coord1, coord2):
    return np.linalg.norm(coord1 - coord2)
dimension=32
def get_charge(symbol):
    return elements[symbol]

def coulomb_matrix(molecule):
    num_atoms = len(molecule)
    coords = np.array([[float(atom[1]), float(atom[2]), float(atom[3])] for atom in molecule])
    charges = np.array([get_charge(atom[0]) for atom in molecule])
    
    coulomb_mat = np.zeros((dimension, dimension))
    
    for i in range(num_atoms):
        for j in range(num_atoms):
            if i == j:
                coulomb_mat[i, j] = 0.5 * charges[i] ** 2.4
            else:
                coulomb_mat[i, j] = charges[i] * charges[j] / distance(coords[i], coords[j])
    
    return coulomb_mat




# Generate Coulomb matrix for water molecule
water_coulomb_mat = coulomb_matrix(mols["water"])

print("Coulomb Matrix for water molecule:")
print(water_coulomb_mat)


Coulomb Matrix for water molecule:
[[73.51669472  8.35268725  8.35268725 ...  0.          0.
   0.        ]
 [ 8.35268725  0.5         0.66032752 ...  0.          0.
   0.        ]
 [ 8.35268725  0.66032752  0.5        ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [11]:
vec = water_coulomb_mat.flatten()

In [12]:
np.set_printoptions(threshold=np.inf)
#print(vec)
#print(type(vec))

In [13]:
mols_vec = {}
for item in mols.items():
    mols_vec[item[0]] = coulomb_matrix(item[1]).flatten() 

In [14]:
#print(mols_vec['water'])

In [15]:
e_diff = pbe0['Energy'].astype(float)-lda['Energy'].astype(float)

In [16]:
#print(mols_vec.keys())
#print(pbe0['Unnamed: 0'])
#print(lda['Unnamed: 0'])

In [17]:
count = 0
name, vector, energy_diff = [], [], []
for i in pbe0['Unnamed: 0']:
    #print(i, mols_vec[i], e_diff[count])
    name.append(i)
    vector.append(mols_vec[i])
    energy_diff.append(e_diff[count])
    count += 1

In [18]:
temp = {'name':name, 'vector':vector, "ediff":energy_diff}
save_data = pd.DataFrame(temp)

In [19]:
save_data.head()

,name,vector,ediff
0,bicyclo[3.1.0]hexane,"[36.85810519942594, 24.365231184761182, 24.365...",-1.907510
1,phosphine,"[332.3449056445624, 10.555930493733829, 10.556...",-1.156955
2,beryllium_sulfide,"[13.928809012737984, 36.74992822279644, 0.0, 0...",-1.395603
3,lithium_oxide,"[73.51669471981023, 14.216325079966829, 0.0, 0...",-0.585126
4,carbon_monoxide,"[36.85810519942594, 42.5418771603297, 0.0, 0.0...",-0.759815


In [20]:
#save_data.to_csv('INPUT1_pbe0_lda.csv', index=False)